In [ ]:
# Cell 1 – Imports, Setup & Utilities
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image as PILImage

import os
from tqdm import tqdm
import seaborn as sns
from datetime import datetime

np.random.seed(42)

# Directory setup
IMG_DIR = "/home/rohitha/ass3"

def create_coord_grid(h, w):
    """
    Create a normalized 2D coordinate grid in [-1, 1]^2.
    
    Parameters
    ----------
    h, w : int
        Height and width of the image.

    Returns
    -------
    coords : np.ndarray
        Array of shape (h*w, 2), each row is [x, y].
    """
    ys, xs = np.linspace(-1, 1, h), np.linspace(-1, 1, w)
    grid_x, grid_y = np.meshgrid(xs, ys)
    coords = np.stack([grid_x, grid_y], axis=-1).reshape(-1, 2)
    return coords


def load_image(path, mode="L", size=(256, 256)):
    """
    Load and resize image from given path.
    
    Parameters
    ----------
    path : str
        Path to image.
    mode : str
        'L' for grayscale or 'RGB' for color.
    size : tuple
        Desired image size.
    
    Returns
    -------
    np.ndarray
        Image array normalized to [0, 1].
    """
    img = PILImage.open(path).convert(mode).resize(size)
    arr = np.asarray(img, dtype=np.float32) / 255.0
    return arr


In [ ]:
# Cell 2 – Feature Mapping Classes
import numpy as np

class BaseFeatureMapping:
    """Abstract base class for all feature mappings."""

    def transform(self, coords: np.ndarray) -> np.ndarray:
        """Transform input coordinates into mapped features."""
        raise NotImplementedError("Subclasses must implement transform().")


class RawMapping(BaseFeatureMapping):
    """Raw coordinate mapping: γ_raw(x, y) = (x, y)"""

    def transform(self, coords: np.ndarray) -> np.ndarray:
        """
        Use raw (x, y) coordinates as features.

        Parameters
        ----------
        coords : np.ndarray
            Normalized coordinates (N, 2).
        
        Returns
        -------
        np.ndarray
            Same as input (N, 2).
        """
        return coords


class PolynomialMapping(BaseFeatureMapping):
    """Polynomial expansion (Taylor-inspired) up to a given order."""

    def __init__(self, order: int = 5):
        self.order = order

    def transform(self, coords: np.ndarray) -> np.ndarray:
        """
        Expand coordinates into polynomial terms:
        [x, y, x², y², xy, ..., x^k, y^k].

        Parameters
        ----------
        coords : np.ndarray
            Normalized coordinates (N, 2).

        Returns
        -------
        np.ndarray
            Expanded feature matrix (N, D).
        """
        x, y = coords[:, 0], coords[:, 1]
        features = [x, y]

        for i in range(2, self.order + 1):
            features += [x ** i, y ** i, (x * y) ** (i - 1)]

        return np.stack(features, axis=1)


class FourierMapping(BaseFeatureMapping):
    """Fourier feature mapping using sin/cos embeddings."""

    def __init__(self, freq: int = 10):
        self.freq = freq

    def transform(self, coords: np.ndarray) -> np.ndarray:
        """
        Apply Fourier mapping to coordinates:
        [1, sin(2πfx), cos(2πfx), sin(2πfy), cos(2πfy)] for f = 1...k

        Parameters
        ----------
        coords : np.ndarray
            Normalized coordinates (N, 2).

        Returns
        -------
        np.ndarray
            Fourier-encoded features (N, D).
        """
        x, y = coords[:, 0], coords[:, 1]
        features = [np.ones_like(x)]  # bias term

        for f in range(1, self.freq + 1):
            features += [
                np.sin(2 * np.pi * f * x),
                np.cos(2 * np.pi * f * x),
                np.sin(2 * np.pi * f * y),
                np.cos(2 * np.pi * f * y),
            ]

        return np.stack(features, axis=1)


In [ ]:
# Cell 3 – Normalization Utilities
def normalize_features(features: np.ndarray, method: str) -> np.ndarray:
    """
    Normalize feature vectors according to the mapping type.
    
    Parameters
    ----------
    features : np.ndarray
        Input features (N, D).
    method : str
        'Raw', 'Polynomial', or 'Fourier'.
    
    Returns
    -------
    np.ndarray
        Normalized feature matrix (N, D).
    """
    method = method.lower()

    if method == "raw":
        normed = (features + 1.0) / 2.0  # scale from [-1, 1] to [0, 1]

    elif method == "polynomial":
        mean = np.mean(features, axis=0, keepdims=True)
        std = np.std(features, axis=0, keepdims=True) + 1e-8
        normed = np.clip((features - mean) / std, -3, 3)

    elif method == "fourier":
        normed = features  # already bounded in [-1, 1]

    else:
        raise ValueError(f"Unknown method: {method}")

    return normed


In [ ]:
# Cell 4 – Modular DataLoader
class ModularDataloader:
    """
    Modular dataloader for preparing image reconstruction datasets
    with different feature mappings (Raw / Polynomial / Fourier).
    """

    def __init__(self, img_path, image_type="Gray", method="Raw", order=5, freq=10):
        """
        Initialize the dataloader.

        Parameters
        ----------
        img_path : str
            Path to the image file.
        image_type : str
            'Gray' or 'RGB'.
        method : str
            Feature mapping type ('Raw', 'Polynomial', 'Fourier').
        order : int
            Order of polynomial expansion (for 'Polynomial').
        freq : int
            Number of Fourier frequencies (for 'Fourier').
        """
        self.img_path = img_path
        self.image_type = image_type
        self.method = method
        self.order = order
        self.freq = freq

        # Step 1: Load image
        mode = "L" if image_type.lower() == "gray" else "RGB"
        self.img = load_image(img_path, mode=mode, size=(256, 256))

        # Step 2: Create coordinate grid
        h, w = self.img.shape[:2]
        self.coords = create_coord_grid(h, w)

        # Step 3: Select feature mapper
        if method == "Raw":
            self.mapper = RawMapping()
        elif method == "Polynomial":
            self.mapper = PolynomialMapping(order=order)
        elif method == "Fourier":
            self.mapper = FourierMapping(freq=freq)
        else:
            raise ValueError("Invalid method. Choose Raw / Polynomial / Fourier.")

        # Step 4: Apply mapping
        raw_features = self.mapper.transform(self.coords)

        # Step 5: Normalize features
        self.features = normalize_features(raw_features, method)

        # Step 6: Prepare target pixels
        self.targets = (
            self.img.reshape(-1, 1)
            if image_type.lower() == "gray"
            else self.img.reshape(-1, 3)
        )

    def get_data(self):
        """
        Returns normalized input features and pixel targets.

        Returns
        -------
        X : np.ndarray
            Normalized input features (N, D)
        y : np.ndarray
            Target pixel values (N, C)
        """
        return self.features, self.targets

    def summary(self):
        """Print dataset details."""
        print(f"Image: {os.path.basename(self.img_path)}")
        print(f"Type: {self.image_type}")
        print(f"Mapping: {self.method}")
        print(f"Feature Dimension: {self.features.shape[1]}")
        print(f"Total Pixels: {self.features.shape[0]}")


In [ ]:
# Cell 5 – Visualization Helpers
def visualize_feature_mapping(mapping, coords, title="Feature Mapping", n_features=3):
    """
    Visualize the first few dimensions of a transformed feature space.

    Parameters
    ----------
    mapping : BaseFeatureMapping
        Instance of RawMapping, PolynomialMapping, or FourierMapping.
    coords : np.ndarray
        Normalized coordinates (N, 2).
    title : str
        Plot title.
    n_features : int
        Number of feature dimensions to visualize.
    """
    transformed = mapping.transform(coords)
    h = w = int(np.sqrt(coords.shape[0]))

    plt.figure(figsize=(15, 4))
    for i in range(min(n_features, transformed.shape[1])):
        plt.subplot(1, n_features, i + 1)
        plt.imshow(transformed[:, i].reshape(h, w), cmap="viridis")
        plt.title(f"{title}\nFeature {i+1}")
        plt.axis("off")
    plt.tight_layout()
    plt.show()


In [ ]:
# Cell 6 – Test Dataloader and Visualizations
# Image paths
smiley_path = os.path.join(IMG_DIR, "smiley.png")
cat_path = os.path.join(IMG_DIR, "cat.jpg")

# Initialize dataloaders
gray_loader = ModularDataloader(smiley_path, "Gray", "Polynomial", order=5)
rgb_loader = ModularDataloader(cat_path, "RGB", "Fourier", freq=10)

# Print summaries
gray_loader.summary()
rgb_loader.summary()

# Visualize targets
plt.figure(figsize=(8, 3))
plt.subplot(1, 2, 1)
plt.imshow(gray_loader.targets.reshape(256, 256), cmap="gray")
plt.title("Grayscale Target Image")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(rgb_loader.targets.reshape(256, 256, 3))
plt.title("RGB Target Image")
plt.axis("off")
plt.tight_layout()
plt.show()

# Visualize feature mappings
coords = create_coord_grid(256, 256)
visualize_feature_mapping(RawMapping(), coords, "Raw Mapping", 2)
visualize_feature_mapping(PolynomialMapping(order=5), coords, "Polynomial Mapping", 3)
visualize_feature_mapping(FourierMapping(freq=10), coords, "Fourier Mapping", 3)


# Imported from 1.ipynb

In [ ]:
import numpy as np
import os
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from IPython.display import display, Image


In [ ]:
class ReLU:
    """ReLU Activation"""
    def forward(self, x):
        self.out = np.maximum(0, x)
        return self.out

    def backward(self, grad_output):
        grad_input = grad_output * (self.out > 0)
        return grad_input

class Tanh:
    """Tanh Activation"""
    def forward(self, x):
        self.out = np.tanh(x)
        return self.out

    def backward(self, grad_output):
        grad_input = grad_output * (1 - self.out**2)
        return grad_input

class Sigmoid:
    """Sigmoid Activation"""
    def forward(self, x):
        self.out = 1 / (1 + np.exp(-x))
        return self.out

    def backward(self, grad_output):
        grad_input = grad_output * self.out * (1 - self.out)
        return grad_input

class Identity:
    def __init__(self):
        pass

    def forward(self, x):
        self.input = x
        return x

    def backward(self, grad_output):
        return grad_output

    def update(self, lr):
        pass

    def zero_grad(self):
        pass  # nothing to reset, but must exist for consistency

    def __repr__(self):
        return "Identity()"


In [ ]:
class Linear:
    """Fully Connected Layer"""
    def __init__(self, in_features, out_features, activation):
        self.in_features = in_features
        self.out_features = out_features
        self.activation = activation

        # Initialize weights and biases
        self.W = np.random.randn(in_features, out_features) * np.sqrt(2.0 / in_features)
        self.b = np.zeros((1, out_features))

        # Cumulative gradients
        self.dW_cum = np.zeros_like(self.W)
        self.db_cum = np.zeros_like(self.b)

    def forward(self, x):
        self.input = x  # Save for backward
        self.linear_out = x @ self.W + self.b
        self.out = self.activation.forward(self.linear_out)
        return self.out

    def backward(self, grad_output):
        # Gradient w.r.t activation
        grad_activation = self.activation.backward(grad_output)
        # Gradients w.r.t weights and biases
        self.dW_cum += self.input.T @ grad_activation
        self.db_cum += np.sum(grad_activation, axis=0, keepdims=True)
        # Gradient w.r.t input for previous layer
        grad_input = grad_activation @ self.W.T
        return grad_input

    def zero_grad(self):
        self.dW_cum.fill(0)
        self.db_cum.fill(0)

    def update(self, lr=0.01):
        self.W -= lr * self.dW_cum
        self.b -= lr * self.db_cum
        self.zero_grad()


In [ ]:
class Model:
    """Neural Network Model"""
    def __init__(self, layers, loss_type="MSE"):
        self.layers = layers
        self.loss_type = loss_type

    def forward(self, x):
        out = x
        for layer in self.layers:
            out = layer.forward(out)
        return out

    def backward(self, grad):
        for layer in reversed(self.layers):
            grad = layer.backward(grad)

    def train(self, x, y):
        """Forward + backward pass, returns scalar loss"""
        y_pred = self.forward(x)
        # Compute loss
        if self.loss_type == "MSE":
            loss = np.mean((y_pred - y) ** 2)
            grad_loss = 2 * (y_pred - y) / y.shape[0]
        elif self.loss_type == "BCE":
            eps = 1e-9
            y_pred = np.clip(y_pred, eps, 1 - eps)
            loss = -np.mean(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))
            grad_loss = (y_pred - y) / (y_pred * (1 - y_pred)) / y.shape[0]
        else:
            raise ValueError("Unknown loss type")

        self.backward(grad_loss)
        return float(loss)

    def zero_grad(self):
        for layer in self.layers:
            layer.zero_grad()

    def update(self, lr=0.01):
        for layer in self.layers:
            layer.update(lr=lr)

    def predict(self, x):
        return self.forward(x)

    def save_to(self, path):
        data = {}
        for idx, layer in enumerate(self.layers):
            # only save layers that have weights
            if hasattr(layer, "W") and hasattr(layer, "b"):
                data[f"W_{idx}"] = layer.W
                data[f"b_{idx}"] = layer.b
        np.savez(path, **data)

    def load_from(self, path):
        loaded = np.load(path)
        for idx, layer in enumerate(self.layers):
            w_key = f"W_{idx}"
            b_key = f"b_{idx}"
            if w_key not in loaded or b_key not in loaded:
                raise ValueError("Architecture mismatch!")
            if layer.W.shape != loaded[w_key].shape or layer.b.shape != loaded[b_key].shape:
                raise ValueError("Shape mismatch!")
            layer.W = loaded[w_key]
            layer.b = loaded[b_key]


# 2.4

In [ ]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image as PILImage
import imageio

# Assume the previous helper classes (Linear, ReLU, Sigmoid, Model, Modular_Dataloader)
# are defined in a separate file or a previous cell.

# --- Configuration ---
# Create a directory to save images, GIFs, and results
IMG_DIR = "training_results"
os.makedirs(IMG_DIR, exist_ok=True)

# Define paths to the images
smiley_path = "smiley.png"
cat_path = "cat.jpg"

In [ ]:
def create_model(input_dim, output_dim, hidden_sizes=[64, 128, 128]):
    """
    Creates a 3-layer MLP model with a consistent architecture.
    
    This function directly supports the assignment's requirement to use the
    same MLP architecture for all methods to ensure a fair comparison.

    Args:
        input_dim (int): The number of input features. This changes based
                         on the feature mapping (Raw, Poly, Fourier).
        output_dim (int): The number of output values (1 for grayscale, 3 for RGB).
        hidden_sizes (list): A list of integers for the sizes of hidden layers.

    Returns:
        Model: An instance of the MLP model.
    """
    layers = [
        Linear(input_dim, hidden_sizes[0], ReLU()),
        Linear(hidden_sizes[0], hidden_sizes[1], ReLU()),
        Linear(hidden_sizes[1], hidden_sizes[2], ReLU()),
        Linear(hidden_sizes[2], output_dim, Sigmoid())
    ]
    return Model(layers, loss_type="MSE")

def train_and_save_epochs(model, X_train, y_train, num_epochs, img_shape, save_dir, batch_size=256, lr=0.1):
    """
    Trains the model and saves the reconstructed image at each epoch.
    
    This function fulfills the core training requirement. It saves the output
    image from each epoch, which is necessary for creating the final comparison GIF.

    Args:
        model (Model): The MLP model to be trained.
        X_train (np.ndarray): The input training data (features).
        y_train (np.ndarray): The target training data (pixel values).
        num_epochs (int): The total number of epochs for training.
        img_shape (tuple): The shape of the output image (e.g., (H, W) or (H, W, C)).
        save_dir (str): The directory where epoch images will be saved.
        batch_size (int): The size of each training batch.
        lr (float): The learning rate for the optimizer.

    Returns:
        tuple: A tuple containing:
            - list: The history of average loss per epoch.
            - list: The history of time taken per epoch.
            - float: The final inference loss on the full dataset.
    """
    os.makedirs(save_dir, exist_ok=True)
    num_samples = X_train.shape[0]
    loss_history = []
    epoch_times = []

    for epoch in range(num_epochs):
        epoch_start = time.time()
        
        # Shuffle data for each epoch
        indices = np.random.permutation(num_samples)
        X_shuffled, y_shuffled = X_train[indices], y_train[indices]
        
        epoch_loss = 0.0
        num_batches = int(np.ceil(num_samples / batch_size))
        
        for i in range(0, num_samples, batch_size):
            X_batch, y_batch = X_shuffled[i:i+batch_size], y_shuffled[i:i+batch_size]
            batch_loss = model.train(X_batch, y_batch)
            model.update(lr=lr)
            epoch_loss += batch_loss
        
        avg_loss = epoch_loss / num_batches
        loss_history.append(avg_loss)
        
        epoch_time = time.time() - epoch_start
        epoch_times.append(epoch_time)
        
        # Save the reconstructed image for this epoch
        y_pred = model.predict(X_train)
        img_recon = np.clip(y_pred.reshape(img_shape), 0, 1)
        img_save = (img_recon * 255).astype(np.uint8)
        
        mode = 'L' if len(img_shape) == 2 else 'RGB'
        PILImage.fromarray(img_save, mode=mode).save(os.path.join(save_dir, f'epoch_{epoch:03d}.png'))
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.6f}, Time: {epoch_time:.2f}s")
            
    final_loss = np.mean((model.predict(X_train) - y_train) ** 2)
    return loss_history, epoch_times, final_loss

In [ ]:
# --- Training Configuration ---
training_configs = {
    "smiley": {"path": smiley_path, "epochs": 50, "shape": (256, 256), "type": "Gray", "output_dim": 1},
    "cat": {"path": cat_path, "epochs": 150, "shape": (256, 256, 3), "type": "RGB", "output_dim": 3}
}
feature_configs = {
    "Raw": [{}],
    "Polynomial": [{"order": o} for o in [5, 15, 25]],
    "Fourier": [{"freq": f} for f in [5, 15, 25]]
}

all_results = []
saved_data = {} # To store loss histories and save directories for GIF creation

# --- Main Training Loop ---
for img_name, img_conf in training_configs.items():
    print(f"\n{'='*80}\nTRAINING ON {img_name.upper()} IMAGE\n{'='*80}")
    saved_data[img_name] = {}
    
    for method, params_list in feature_configs.items():
        for params in params_list:
            param_str = ", ".join([f"{k}={v}" for k, v in params.items()])
            print(f"\n--- Training with {method} Features ({param_str or 'Baseline'}) ---")
            
            # 1. Load Data
            loader = ModularDataloader(img_path=img_conf["path"], image_type=img_conf["type"], method=method, **params)
            X_train, y_train = loader.get_data()
            
            # 2. Create Model
            model = create_model(input_dim=X_train.shape[1], output_dim=img_conf["output_dim"])
            
            # 3. Train Model
            save_dir_name = f"results_{img_name}_{method.lower()}" + "".join([f"_{v}" for v in params.values()])
            save_dir = os.path.join(IMG_DIR, save_dir_name)
            
            loss_hist, time_hist, final_loss = train_and_save_epochs(
                model, X_train, y_train,
                num_epochs=img_conf["epochs"],
                img_shape=img_conf["shape"],
                save_dir=save_dir
            )
            
            # 4. Store Results
            result = {
                'Image': img_name.capitalize(),
                'Method': method,
                'Parameters': param_str or '-',
                'Final Loss': final_loss,
                'Avg Epoch Time (s)': np.mean(time_hist),
                'Input Dimensions': X_train.shape[1]
            }
            all_results.append(result)
            
            # Save data needed for GIF
            key = f"{method}_{param_str}" if params else method
            saved_data[img_name][key] = {'loss_history': loss_hist, 'save_dir': save_dir}

            print(f"Final Loss: {final_loss:.6f}, Avg Epoch Time: {np.mean(time_hist):.2f}s")

In [ ]:
# Create a DataFrame from the collected results
df_results = pd.DataFrame(all_results)

# Display table for Smiley
print(f"\n{'='*80}\nRESULTS TABLE - SMILEY IMAGE\n{'='*80}")
df_smiley = df_results[df_results['Image'] == 'Smiley'].drop(columns='Image')
print(df_smiley.to_string(index=False))
df_smiley.to_csv(os.path.join(IMG_DIR, 'results_smiley.csv'), index=False)

# Display table for Cat
print(f"\n{'='*80}\nRESULTS TABLE - CAT IMAGE\n{'='*80}")
df_cat = df_results[df_results['Image'] == 'Cat'].drop(columns='Image')
print(df_cat.to_string(index=False))
df_cat.to_csv(os.path.join(IMG_DIR, 'results_cat.csv'), index=False)

In [ ]:
def create_comparison_gif(raw_data, poly_data, fourier_data, poly_label, fourier_label, num_epochs, output_path, title_prefix=""):
    """
    Creates a 1x3 subplot GIF comparing Raw, Polynomial, and Fourier methods.
    
    This function directly addresses the visualization requirement by generating
    a GIF that includes a legend in each subplot showing the live loss curve.

    Args:
        raw_data (dict): Dict with 'loss_history' and 'save_dir' for Raw features.
        poly_data (dict): Dict for the chosen Polynomial features.
        fourier_data (dict): Dict for the chosen Fourier features.
        poly_label (str): Label for the Polynomial subplot (e.g., "Poly (Order=15)").
        fourier_label (str): Label for the Fourier subplot (e.g., "Fourier (Freq=15)").
        num_epochs (int): The total number of epochs (frames in the GIF).
        output_path (str): The file path to save the generated GIF.
        title_prefix (str): A prefix for the main GIF title (e.g., "Smiley").
    """
    gif_frames = []
    all_losses = raw_data['loss_history'] + poly_data['loss_history'] + fourier_data['loss_history']
    min_loss, max_loss = np.min(all_losses), np.max(all_losses)

    for epoch in range(num_epochs):
        fig, axes = plt.subplots(1, 3, figsize=(18, 6.5))
        fig.suptitle(f'{title_prefix} Reconstruction Comparison (Epoch {epoch+1}/{num_epochs})', fontsize=16)

        # --- Data for the 3 subplots ---
        plot_data = [
            {'title': 'Raw Features', 'data': raw_data},
            {'title': poly_label, 'data': poly_data},
            {'title': fourier_label, 'data': fourier_data},
        ]

        for i, p in enumerate(plot_data):
            # Load the main image
            img_path = os.path.join(p['data']['save_dir'], f'epoch_{epoch:03d}.png')
            img = PILImage.open(img_path)
            axes[i].imshow(img, cmap='gray' if img.mode == 'L' else None)
            axes[i].set_title(f"{p['title']}\nLoss: {p['data']['loss_history'][epoch]:.5f}", fontsize=12)
            axes[i].axis('off')
            
            # --- Add inset plot for the loss curve ---
            ax_inset = axes[i].inset_axes([0.05, 0.05, 0.35, 0.25], facecolor='w', alpha=0.8)
            ax_inset.plot(range(epoch + 1), p['data']['loss_history'][:epoch + 1], color='b')
            ax_inset.set_yscale('log')
            ax_inset.set_ylim(min_loss, max_loss) # Consistent y-axis
            ax_inset.set_title("Loss Curve", fontsize=8)
            ax_inset.tick_params(axis='both', which='major', labelsize=7)
            ax_inset.grid(True, alpha=0.4)
            
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        
        # Convert plot to an image array for the GIF
        fig.canvas.draw()
        frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
        frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        gif_frames.append(frame)
        plt.close(fig)

    # Save the frames as a GIF
    imageio.mimsave(output_path, gif_frames, fps=5, loop=0)
    print(f"GIF saved successfully to: {output_path}")

In [ ]:
from IPython.display import Image as IPyImage, display

# --- Choose one combination of features for the GIF ---
# We'll pick the mid-range parameters for a representative comparison.
POLY_ORDER_GIF = 15
FOURIER_FREQ_GIF = 15

# --- Generate Smiley GIF ---
print("\nCreating GIF for SMILEY image comparison...")
smiley_gif_path = os.path.join(IMG_DIR, 'smiley_comparison.gif')
create_comparison_gif(
    raw_data=saved_data['smiley']['Raw'],
    poly_data=saved_data['smiley'][f'Polynomial_order={POLY_ORDER_GIF}'],
    fourier_data=saved_data['smiley'][f'Fourier_freq={FOURIER_FREQ_GIF}'],
    poly_label=f'Polynomial (Order={POLY_ORDER_GIF})',
    fourier_label=f'Fourier (Freq={FOURIER_FREQ_GIF})',
    num_epochs=training_configs['smiley']['epochs'],
    output_path=smiley_gif_path,
    title_prefix="Smiley"
)
display(IPyImage(filename=smiley_gif_path))


# --- Generate Cat GIF ---
print("\nCreating GIF for CAT image comparison...")
cat_gif_path = os.path.join(IMG_DIR, 'cat_comparison.gif')
create_comparison_gif(
    raw_data=saved_data['cat']['Raw'],
    poly_data=saved_data['cat'][f'Polynomial_order={POLY_ORDER_GIF}'],
    fourier_data=saved_data['cat'][f'Fourier_freq={FOURIER_FREQ_GIF}'],
    poly_label=f'Polynomial (Order={POLY_ORDER_GIF})',
    fourier_label=f'Fourier (Freq={FOURIER_FREQ_GIF})',
    num_epochs=training_configs['cat']['epochs'],
    output_path=cat_gif_path,
    title_prefix="Cat"
)
display(IPyImage(filename=cat_gif_path))